<a href="https://colab.research.google.com/github/ItsShi/Mono-Depth/blob/main/squared.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install timm tensorboardX==1.4 torchvision==0.12.0

In [ ]:
!rm -rf /content/asdf
!git clone https://github.com/ItsShi/asdf.git

for opt in ["train", "val", "test"]:
    f = open("/content/asdf/splits/endovis/{}_files.txt".format(opt), "r+")
    # f = open("/content/{}_files_original.txt".format(opt), "r+")
    new_file = []
    for line in f:
      folder, frame = line.split()[0], line.split()[1]
      if int(frame)<20 and int(frame)>0:
        if folder == "dataset3/keyframe1" or folder == "dataset3/keyframe3" or folder == "dataset3/keyframe4":
          new_file.append(line)
    print(len(new_file))
    with open("{}_files.txt".format(opt), "w+") as f:
      for i in new_file:
        f.write(i)

    !cp -f /content/{opt}_files.txt /content/asdf/splits/endovis/{opt}_files.txt

In [ ]:
# import gdown

# perfect_gt_url = 'https://drive.google.com/uc?id=1GHX4ML7kUP4xFZSi56VtpZSxZdwHa2oO'
# gdown.download(perfect_gt_url,'perfect_gt.zip',quiet=True)
# !unzip -q perfect_gt.zip

import gdown

!mkdir videos
d3k1_url = 'https://drive.google.com/uc?id=17k1-CHptG_XTXVX8qp7yK_zadH54vVnY'
gdown.download(d3k1_url,'videos/d3k1_rgb.mp4',quiet=True)
# frame=  329 fps= 23 q=24.8 Lsize=N/A time=00:00:13.16 bitrate=N/A speed=0.904x

!pip install ffmpeg

video_name = "d3k1"
!ffmpeg -i /content/videos/{video_name}_rgb.mp4 -filter:v "crop=1280:1024:0:0" /content/videos/{video_name}_crop_rgb.mp4
!ffmpeg -i /content/videos/{video_name}_crop_rgb.mp4 %6d.jpg
!mkdir -p /content/frames/{video_name}
!mv *.jpg /content/frames/{video_name}

import PIL.Image as pil
im = pil.open("/content/frames/{}/000001.jpg".format(video_name))
print(im.size)

In [ ]:
!python asdf/train_end_to_end.py --no_cuda --data_path frames/d3k1 --log_dir asdf  --num_epochs 1 --batch_size 1 --width 224 --height 224 --scales 0 # --scales 0 1 2  #--load_weights_folder womin/mdp/models/weights_19  --models_to_load "encoder" "depth" "pose_encoder" "pose"